#### EDA Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
%matplotlib inline

data source url 

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

`let's fetch data using requests library`

In [3]:
response = requests.get(url)
print("Response Code: ", response)
# response Code 200 all good

Response Code:  <Response [200]>


### Let's Scrap Data from Website usin Beautiful Soup

In [4]:
import bs4 

In [5]:
soup = bs4.BeautifulSoup(response.content, 'html.parser')
# parsing html data to create a beautiful soup object

#### Let's Find Tables in this page

In [6]:
print(len(soup.find_all('table')))
# total of 3 tables are there on this page

3


## Let's Access data cells in first table:

    some how i looked on all tables and find only first table is suitable for our purpose lo let's dive in

In [7]:
Postal_Code = []
# to store postal code info
Borough = []
# to store borough info
Neighbourhood = []
# to store neighborhood info

In [8]:
for row in soup.find('table').find_all('tr'):
    # iterating through each row in first table
    cells = row.find_all('td')
    # getting all columns of first row
    if len(cells) > 0: # some time we have empty and heading columns so it's good idea to check it
        Postal_Code.append(cells[0].text.strip()) # 0th column has Postal Code Info
        Borough.append(cells[1].text.strip())# 1st column has Borough Information
        Neighbourhood.append(cells[2].text.strip()) # 2nd Column has Neighborhood Information

### `3.1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood`

In [9]:
# creating data frame for better representation and easy data pre-processing purpose
data = pd.DataFrame({
    "PostalCode": Postal_Code,
    "Borough": Borough,
    "Neighborhood": Neighbourhood
})

In [10]:
data.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### let's look into data a little bit

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
PostalCode      180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


#### We have total 180 Entries

### let's start with Data Exploring

### `3.2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.`

#### check out all cells with Not assigned Borough

In [12]:
data[data['Borough'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
7,M8A,Not assigned,Not assigned
10,M2B,Not assigned,Not assigned
15,M7B,Not assigned,Not assigned
...,...,...,...
174,M4Z,Not assigned,Not assigned
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned


`we have 77 rows out of 180 having No Borough Information in them`

### Dropping all Cells having Bourough "Not assigned"

In [13]:
data = data[data['Borough'] != 'Not assigned'].reset_index(drop=True)

data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### Let's check if some PostalCode and Boroguh are redudent

In [14]:
(data['PostalCode'].value_counts() > 1).sum()

0

In [15]:
(data["Borough"].value_counts() > 1).sum()

9

`There are total 9 unique Borough Exists in Table`

In [16]:
data["Borough"].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64

`Let's Group all neighborshood in the same borough`

### `3.3 More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.`

In [17]:
df = data.groupby(['PostalCode', 'Borough'], as_index=False).agg(lambda x: ",".join(x))
# joining all neighborhoods which share same postal code and borough

In [18]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
df.shape

(103, 3)

### `3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.`

let's check if some shell does not have neighborhood

In [20]:
(df['Neighborhood'] == 'Not assigned').sum()

0

In [21]:
(df['Borough'] == 'Not assigned').sum()

0

`It seems we does not have any Not assigned shell in both columns but it's good to check and write code to replace all negihbors with borough if neighbors have value Not assigned`

In [22]:
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
        
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


`Final shape of Our Data`

In [24]:
df.shape

(103, 3)

#### Let's verify is our DataFrame same as required by the question

In [25]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

index_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in index_list:
    test_df = test_df.append(df[df["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


___
Week 3 Part 1 Finished Here
___

### `Load Location Coordinates from the CSV`

In [26]:
lat_lon_df = pd.read_csv('Geospatial_Coordinates.csv')
# provided by Coursera

In [27]:
lat_lon_df.columns  = [ 'PostalCode', 'Latitude', 'Longitude']

In [28]:
lat_lon_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Let's Merge coordicated data with our neighbours data

In [29]:
df.shape

(103, 3)

In [30]:
lat_lon_df.shape

(103, 3)

`we have same shapes`

In [31]:
location_df = df.merge(lat_lon_df, on='PostalCode', how='left')

In [32]:
location_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [33]:
location_df.shape

(103, 5)

In [34]:
location_df.isna().sum()

PostalCode      0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

In [35]:
location_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
Latitude        103 non-null float64
Longitude       103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


`No NULL values present in data that's great`

`Let's print coordinates according to question`

In [36]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(location_df[location_df["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


___
Week3 Part2 Over Here
___

#### Importing Clustering and GeoSpatial Data Visulation Library

In [37]:
import folium

from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim

#### `Let's Collect latitude and longitude values for Toronto Using GeoLocator`

In [38]:
address = "Toronto"

In [39]:
geolocator = Nominatim(user_agent='application')
location = geolocator.geocode(address)

In [40]:
lat = location.latitude

In [41]:
long = location.longitude

In [42]:
print(f"Coordinates of Toronto are {lat}, {long}")

Coordinates of Toronto are 43.6534817, -79.3839347


### Let's Create a map of Toronto

In [43]:
map_toronto = folium.Map(location=[lat, long], zoom_start=10)

In [44]:
map_toronto

 Toronto is on beach Nice Location

### Mapping Toronto Map with neighborhoods superimposed on top

In [47]:
for borough, neighborhood, lt, lng in location_df[[  'Borough', 'Neighborhood','Latitude', 'Longitude']].values:
    popup = folium.Popup(f'{neighborhood}, {borough}', parse_html=True)
    folium.CircleMarker(
        [lt, lng],
        radius=5,
        popup=popup,
        color='gold',
        fill=True,
        fill_color='salmon',
        fill_opacity=0.75
    
    ).add_to(map_toronto)

In [48]:
map_toronto

### Let's Filter Borough Which contain word Toronto in Them

In [49]:
toronto_borough = []
for name in location_df.Borough.unique():
    if "toronto" in name.lower():
        toronto_borough.append(name)

In [50]:
toronto_borough

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

### Let's Explore all Neighbours which have Borough Toronto

In [51]:
# let's 
client_id = 'your foursquare api id'
client_secret = 'your foursquare api secret'
v='20180323'

### we will search for radius 500 meter and 100 events
radius = 500
limit = 100

venues = []

In [55]:
for neigh, borough, lt, ln, code in location_df[['Neighborhood', 'Borough', 'Latitude', 'Longitude', 'PostalCode' ]].values:
    url = 'https://api.foursquare.com/v2/venues/explore'
    #Foursquare Endpoints
    params = dict(
        client_id = client_id,
        client_secret = client_secret,
        v=v,
        ll=f"{lt},{ln}",
        limit=limit,
        radius=radius
    )
    # Parameters to find location data of 500 meter
    resp = requests.get(url,params=params)
    
    items = resp.json()["response"]["groups"][0]['items']
    for venue in items:
        venues.append(
                (
                    borough, neighborhood, code, lt, ln, 
                    venue['venue']['name'],
                    venue['venue']['location']['lat'],
                    venue['venue']['location']['lng'],
                    venue['venue']['categories'][0]['name']
                )
    
            )
   

In [57]:
# let's form a dataframe for better preprocessing
venues_df = pd.DataFrame(venues, columns=["Borough", "Neighborhood", "PostalCode",
                                         "Latitude", "Longitude", "VenueName",
                                         "VenueLatitude", "VenueLongitude",
                                         "VenueCategory"])

In [58]:
venues_df.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Scarborough,"Northwest, West Humber - Clairville",M1B,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,"Northwest, West Humber - Clairville",M1B,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Scarborough,"Northwest, West Humber - Clairville",M1C,43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
3,Scarborough,"Northwest, West Humber - Clairville",M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,Scarborough,"Northwest, West Humber - Clairville",M1E,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [59]:
venues_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4068 entries, 0 to 4067
Data columns (total 9 columns):
Borough           4068 non-null object
Neighborhood      4068 non-null object
PostalCode        4068 non-null object
Latitude          4068 non-null float64
Longitude         4068 non-null float64
VenueName         4068 non-null object
VenueLatitude     4068 non-null float64
VenueLongitude    4068 non-null float64
VenueCategory     4068 non-null object
dtypes: float64(4), object(5)
memory usage: 286.2+ KB


In [60]:
venues_df.shape

(4068, 9)

#### Let's check how many venues we have got for each PostalCode

In [61]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [62]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M1B,Scarborough,"Northwest, West Humber - Clairville",4,4,4,4,4,4
M1C,Scarborough,"Northwest, West Humber - Clairville",4,4,4,4,4,4
M1E,Scarborough,"Northwest, West Humber - Clairville",16,16,16,16,16,16
M1G,Scarborough,"Northwest, West Humber - Clairville",8,8,8,8,8,8
M1H,Scarborough,"Northwest, West Humber - Clairville",18,18,18,18,18,18
M1J,Scarborough,"Northwest, West Humber - Clairville",2,2,2,2,2,2
M1K,Scarborough,"Northwest, West Humber - Clairville",12,12,12,12,12,12
M1L,Scarborough,"Northwest, West Humber - Clairville",16,16,16,16,16,16
M1M,Scarborough,"Northwest, West Humber - Clairville",6,6,6,6,6,6


### Unique Categories of Venues

In [63]:
total_categories = venues_df['VenueCategory'].unique().shape[0]
total_categories

269

In [64]:
print(f"We have got {total_categories} Unique Categories for venues")

We have got 269 Unique Categories for venues


#### let's see some of them

In [65]:
print(*map(lambda i:f"{i[0]+1:>2}. {i[1]}", enumerate(venues_df['VenueCategory'].unique()[:20])), sep='\n')

 1. Fast Food Restaurant
 2. Print Shop
 3. Home Service
 4. Bar
 5. Bank
 6. Electronics Store
 7. Restaurant
 8. Mexican Restaurant
 9. Rental Car Location
10. Medical Center
11. Intersection
12. Breakfast Spot
13. Coffee Shop
14. Korean Restaurant
15. Caribbean Restaurant
16. Hakka Restaurant
17. Thai Restaurant
18. Athletics & Sports
19. Bakery
20. Gas Station


## Exploratory Data Analysis

### let's Analyze Each Neighborhood

In [67]:
toronto_onehot= pd.get_dummies(venues_df.VenueCategory)
## one hot encoding for each unique venue type we will have a column

In [68]:
toronto_onehot['PostalCode'] = venues_df.PostalCode
toronto_onehot['Borough'] = venues_df.Borough
toronto_onehot['Neighborhoods'] = venues_df.Neighborhood

In [69]:
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]


In [70]:
toronto_onehot.shape

(4068, 272)

In [71]:
toronto_onehot.head()

,PostalCode,Borough,Neighborhoods,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,Scarborough,"Northwest, West Humber - Clairville",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [73]:
# EDA
toronto_onehot[toronto_onehot.columns[3:]].describe()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
count,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.0000

### Let's Group rows by neighborhood and take mean of frequency for each category

In [74]:
toronto_grouped = toronto_onehot.groupby(['PostalCode', 'Borough', 'Neighborhoods']).mean().reset_index()

In [75]:
toronto_grouped.shape

(98, 272)

In [76]:
location_df.shape

(103, 5)

`ohh ho ho, some of PostalCodes are Missing Let's Find Out Which One Missing`

In [77]:
location_df[~location_df['PostalCode'].isin(toronto_grouped['PostalCode'])]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
20,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
21,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493
93,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
94,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724


In [78]:
toronto_grouped.describe()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
count,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.

### Top 10 Venues for each Postal Code

In [79]:
limit = 10

indicators = ['st', 'nd', 'rd']

areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']

freqColumns = []

for ind in np.arange(limit):
    try:
        freqColumns.append(f'{ind+1}{indicators[ind]} Most Common Venue')
    except:
        freqColumns.append(f"{ind+1}th Most Common Venue")
        
columns = areaColumns+freqColumns

In [80]:
neigh_sorted = pd.DataFrame(columns=columns)

In [81]:
neigh_sorted['PostalCode'] = toronto_grouped['PostalCode']
neigh_sorted['Borough'] = toronto_grouped['Borough']
neigh_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

In [82]:
for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neigh_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:limit]

In [83]:
neigh_sorted.shape

(98, 13)

In [84]:
neigh_sorted

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Northwest, West Humber - Clairville",Fast Food Restaurant,Print Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
1,M1C,Scarborough,"Northwest, West Humber - Clairville",Home Service,Bar,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
2,M1E,Scarborough,"Northwest, West Humber - Clairville",Rental Car Location,Mexican Restaurant,Restaurant,Electronics Store,Breakfast Spot,Bank,Medical Center,Intersection,Distribution Center,Dog Run
3,M1G,Scarborough,"Northwest, West Humber - Clairville",Coffee Shop,Korean Restaurant,Mexican Restaurant,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,M1H,Scarborough,"Northwest, West Humber - Clairville",Hakka Restaurant,Bakery,Lounge,Caribbean Restaurant,Athletics & Sports,Gas Station,Thai Restaurant,Fried Chicken Joint,Bank,Drugstore
5,M1J,Scarborough,"Northwest, West Humber - Clairville",Playground,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
6,M1K,Scarborough,"Northwest, West Humber - Clairville",Department Store,Discount Store,Bus Station,Coffee Shop,Hobby Shop,Chinese Restaurant,Donut Shop,Distribution Center,Dog Run,Doner Restaurant
7,M1L,Scarborough,"Northwest, West Humber - Clairville",Bakery,Bus Line,Park,Metro Station,Ice Cream Shop,Soccer Field,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
8,M1M,Scarborough,"Northwest, West Humber - Clairville",Skating Rink,Motel,American Restaurant,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
9,M1N,Scarborough,"Northwest, West Humber - Clairville",Café,General Entertainment,Skating Rink,Farm,College Stadium,Construction & Landscaping,Convenience Store,Falafel Restaurant,Comfort Food Restaurant,Event Space


# Clustering

#### Lets Cluster Areas using K-Means Algo

In [85]:
k = 5

clusters = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], axis=1)

In [86]:
kmeans_model = KMeans(n_clusters=k, random_state=0)

In [87]:
kmeans_model.fit(clusters)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [88]:
kmeans_model.labels_[:10]

array([3, 3, 3, 3, 3, 2, 3, 3, 3, 3])

#### Let's Merge Clusters as well as the top 10 venues for each neighborhood

In [89]:
merged_df = location_df.copy()[location_df['PostalCode'].isin(toronto_grouped['PostalCode'])]

In [90]:
merged_df['ClusterLabel'] = kmeans_model.labels_

In [91]:
merged_df = merged_df.join(neigh_sorted.drop(["Borough", "Neighborhoods"], axis=1
                                            ).set_index("PostalCode"),
                          on = "PostalCode")

In [92]:
merged_df.shape

(98, 16)

In [93]:
merged_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3,Fast Food Restaurant,Print Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,3,Home Service,Bar,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3,Rental Car Location,Mexican Restaurant,Restaurant,Electronics Store,Breakfast Spot,Bank,Medical Center,Intersection,Distribution Center,Dog Run
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Korean Restaurant,Mexican Restaurant,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Hakka Restaurant,Bakery,Lounge,Caribbean Restaurant,Athletics & Sports,Gas Station,Thai Restaurant,Fried Chicken Joint,Bank,Drugstore
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Playground,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,3,Department Store,Discount Store,Bus Station,Coffee Shop,Hobby Shop,Chinese Restaurant,Donut Shop,Distribution Center,Dog Run,Doner Restaurant
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,3,Bakery,Bus Line,Park,Metro Station,Ice Cream Shop,Soccer Field,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,3,Skating Rink,Motel,American Restaurant,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,3,Café,General Entertainment,Skating Rink,Farm,College Stadium,Construction & Landscaping,Convenience Store,Falafel Restaurant,Comfort Food Restaurant,Event Space


#### Let's Map Clusters to the Map of Toronto

In [94]:
merged_df.sort_values('ClusterLabel', inplace=True)

In [95]:
merged_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304,0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Light Rail Station,Chinese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,Gym,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
96,M9L,North York,Humber Summit,43.756303,-79.565963,1,Furniture / Home Store,Gym,Pizza Place,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Playground,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3,Fast Food Restaurant,Print Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop


In [96]:
cluster_map = folium.Map(
    location = [lat, long], zoom_start=10
    )

In [97]:
cluster_map

### Okay Let's draw Clusters

In [98]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [99]:
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k) ]

c_array = cm.jet(np.linspace(0, 1, len(ys)))

rainbow = [ colors.rgb2hex(i) for i in c_array]


In [100]:
marker_colors = []

In [101]:
for post, borough, neighbor, lat, lon, cls in merged_df[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude',
       'ClusterLabel']].values:
    label = folium.Popup(f"{borough}, ({post}): {'borough'} - Cluster {cls}")
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cls-1],
        fill=True,
        fill_color=rainbow[cls-1],
        fil_opacity=0.7
    ).add_to(cluster_map)


In [102]:
cluster_map

## !! Final Step Examine Clusters

### Cluster 1

In [103]:
merged_df.loc[merged_df['ClusterLabel'] == 0, merged_df.columns[[1]+ list(range(5, merged_df.shape[1] ))]]

,Borough,ClusterLabel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Light Rail Station,Chinese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run


## Cluster 2

In [104]:
merged_df.loc[merged_df['ClusterLabel'] == 1, merged_df.columns[[1]+ list(range(5, merged_df.shape[1] ))]]

,Borough,ClusterLabel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Gym,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
96,North York,1,Furniture / Home Store,Gym,Pizza Place,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


### Cluster 3

In [105]:
merged_df.loc[merged_df['ClusterLabel'] == 2, merged_df.columns[[1]+ list(range(5, merged_df.shape[1] ))]]

,Borough,ClusterLabel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2,Playground,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


### Cluster 4

In [106]:
merged_df.loc[merged_df['ClusterLabel'] == 3, merged_df.columns[[1]+ list(range(5, merged_df.shape[1] ))]]

,Borough,ClusterLabel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3,Fast Food Restaurant,Print Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
70,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Salad Place,Seafood Restaurant,Steakhouse,Asian Restaurant
69,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Beer Bar,Japanese Restaurant,Italian Restaurant,Hotel,Seafood Restaurant,Pub,Park
68,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
67,Downtown Toronto,3,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Bar,Park,Gaming Cafe,Pizza Place,Arts & Crafts Store
66,Downtown Toronto,3,Café,Bar,Japanese Restaurant,Bookstore,Sandwich Place,Restaurant,Bakery,Yoga Studio,Beer Bar,Beer Store
65,Central Toronto,3,Sandwich Place,Café,Coffee Shop,Liquor Store,Pub,Middle Eastern Restaurant,Flower Shop,BBQ Joint,Metro Station,Pizza Place
64,Central Toronto,3,Park,Trail,Sushi Restaurant,Jewelry Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
63,Central Toronto,3,Pool,Garden,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
62,North York,3,Coffee Shop,Sandwich Place,Juice Bar,Italian Restaurant,Hobby Shop,Pharmacy,Pizza Place,Café,Butcher,Pub


### Cluster 5

In [107]:
merged_df.loc[merged_df['ClusterLabel'] == 4, merged_df.columns[[1]+ list(range(5, merged_df.shape[1] ))]]

,Borough,ClusterLabel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
100,Etobicoke,4,Park,Mobile Phone Shop,Sandwich Place,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,College Rec Center
74,York,4,Park,Pool,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
98,York,4,Park,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Farmers Market
14,Scarborough,4,Park,Bakery,Playground,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
44,Central Toronto,4,Park,Swim School,Bus Line,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dessert Shop
23,North York,4,Park,Convenience Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
90,Etobicoke,4,River,Park,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
40,East York,4,Park,Convenience Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
50,Downtown Toronto,4,Park,Trail,Playground,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
25,North York,4,Park,Food & Drink Shop,Pool,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store


### Observations: 

Most of the neighborhoods fall into Cluster 2 which are mostly business area`s with Fast Food Restaurant,	Print Shop,	Donut Shop,	Diner,	Discount Store,	Distribution Center,	Dog Run,	Doner Restaurant,	Drugstore,	Dessert Shop`

Cluster 1 has `Indian Restaurant` at borough Scarborough

Cluster 2 has `Gym and Furniture / Home Store1` around borough Central Toronto and North York

Cluster 3 has `Playground` around borough Scarborough	

Cluster 5 is also have many venues such as `Park, Food Shop, Restaurant etc`